# HHI of Stargaze network

Get the state of all validators.

In [30]:
!starsd query staking validators --node https://rpc.stargaze.ezstaking.io:443/ --output json > validators.json

Get the delegation associated with each validator.

In [37]:
import json
validators = json.load(open('validators.json', 'r'))['validators']
validators[0]
# tokens = [int(v['tokens']) for v in validators]
# tokens

{'operator_address': 'starsvaloper1qz3zj0j3546xywfe8qwcve5flne9w98qu90048',
 'consensus_pubkey': {'@type': '/cosmos.crypto.ed25519.PubKey',
  'key': 'mRscRbPomJJDXnG9R6o/CwvVPvCWqISXPbFskv30p8o='},
 'jailed': False,
 'status': 'BOND_STATUS_BONDED',
 'tokens': '2006079000000',
 'delegator_shares': '2006079000000.000000000000000000',
 'description': {'moniker': '0xSR | 0xSR.com',
  'identity': 'E764DC957E82F233',
  'website': 'https://0xSR.com',
  'security_contact': 'sr@0xsr.com',
  'details': 'You can reach me on my website, which is https://0xSR.com, all the time.'},
 'unbonding_height': '0',
 'unbonding_time': '1970-01-01T00:00:00Z',
 'commission': {'commission_rates': {'rate': '0.050000000000000000',
   'max_rate': '1.000000000000000000',
   'max_change_rate': '0.990000000000000000'},
  'update_time': '2021-11-01T09:41:32.057072748Z'},
 'min_self_delegation': '1'}

In [32]:
def values_to_marketshares (values: List[int]) -> List[float]:
    '''
    Takes value of each firm and turns it into a proportion, i.e., 
    their share of the total market.
    '''
    total = sum(values)
    return [v / total for v in values ]

assert(values_to_marketshares([10, 10, 30]) == [0.2, 0.2, 0.6])

In [33]:
from typing import List

def hhi (marketshares: List[float]):
    '''
    "The HHI is calculated by squaring the market share of each firm competing in the market 
    and then summing the resulting numbers."
    
    U.S. federal agencies "generally consider markets in which the HHI is between 
    1,500 and 2,500 points to be moderately concentrated, and consider markets 
    in which the HHI is in excess of 2,500 points to be highly concentrated."
    
    From https://www.justice.gov/atr/herfindahl-hirschman-index
    '''
    squares = [(x*100)*(x*100) for x in marketshares]
    return sum(squares)
    
assert(hhi([.30, .30, .20, .20]) == 2600)

In [34]:
hhi(values_to_marketshares(tokens))

156.48663784255817